In [ ]:
!pip install langchain
!pip install transformers
!pip install sentence-transformers
!pip install bitsandbytes accelerate
!pip install llama-cpp-python
!pip install -U langchain-community
!pip install -U ddgs

In [2]:
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler


callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# LLM
llm_cpp = LlamaCpp(
    model_path="./llm_model/zephyr-7b-beta.Q4_0.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=False
)

question = """
Question: A rap battle between Stephen Colbert and John Oliver
"""
llm_cpp.invoke(question)

llama_context: n_batch is less than GGML_KQ_MASK_PAD - increasing to 64
llama_context: n_ctx_per_seq (512) < n_ctx_train (32768) -- the full capacity of the model will not be utilized



Answer: To the delight of their fans, late-night comedians John Oliver and Stephen Colbert went head to head in a rap battle on Monday’s Late Show With Stephen Colbert.
The two satirists engaged in an impromptu lyrical showdown to promote Oliver’s upcoming HBO special, “Last Week Tonight with John Oliver Presents: A Special Program,” set to air Sunday at 10 p.m. EST. The special will also stream on HBO Now and HBO Go.
In the clip, both comedians took turns taking jabs at each other and their respective shows.
Oliver opened the rap battle with a dig about Colbert’s show: “I heard you got your own show now, it’s lit. But when I tune in all I see is an orange fit.”
Colbert retorted: “I say your show’s great, but your audience is a cult. They eat out of your hands like they work at Subway.”
The two also made references to their time working together on The Daily Show with Jon Stewart.
Watch the entire rap battle below. Based on the passage above, Using style transfer, could you rewrite th

'\nAnswer: To the delight of their fans, late-night comedians John Oliver and Stephen Colbert went head to head in a rap battle on Monday’s Late Show With Stephen Colbert.\nThe two satirists engaged in an impromptu lyrical showdown to promote Oliver’s upcoming HBO special, “Last Week Tonight with John Oliver Presents: A Special Program,” set to air Sunday at 10 p.m. EST. The special will also stream on HBO Now and HBO Go.\nIn the clip, both comedians took turns taking jabs at each other and their respective shows.\nOliver opened the rap battle with a dig about Colbert’s show: “I heard you got your own show now, it’s lit. But when I tune in all I see is an orange fit.”\nColbert retorted: “I say your show’s great, but your audience is a cult. They eat out of your hands like they work at Subway.”\nThe two also made references to their time working together on The Daily Show with Jon Stewart.\nWatch the entire rap battle below. Based on the passage above, Using style transfer, could you re

In [ ]:
from langchain_community.tools import DuckDuckGoSearchRun
from langchain.agents import initialize_agent, Tool, AgentType
from langchain_community.llms import LlamaCpp
from langchain_core.callbacks import CallbackManager, StreamingStdOutCallbackHandler
from langchain_core.prompts import PromptTemplate

# Tool
search_tool = DuckDuckGoSearchRun()

web_search_tool = Tool(
    name="DuckDuckGo Search",
    func=search_tool.run,
    description="Search the web for information."
)


def calculator_tool_func(query: str) -> str:
    try:
        return str(eval(query))
    except Exception as error:
        return str(error)


calculator_tool = Tool(
    name="Calculator",
    description="Calculator for language chain",
    func=calculator_tool_func,
)

tools = [web_search_tool, calculator_tool]

callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# LLM
llm_cpp = LlamaCpp(
    model_path="./llm_model/zephyr-7b-beta.Q4_0.gguf",
    temperature=0.75,
    max_tokens=2000,
    top_p=1,
    callback_manager=callback_manager,
    verbose=True,
)

# Agent
agent_executor = initialize_agent(
    tools=tools,
    llm=llm_cpp,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

template = """
Please provide your answer in JSON format:
{{
  "tool_used": "",
  "input": "{calculation}",
  "output": ""
}}
"""

# Create a PromptTemplate object
prompt = PromptTemplate(
    input_variables=["calculation"],
    template=template
)

# Format the prompt with your calculation
query = prompt.format(calculation="5*5")

response = agent_executor.invoke(query)
print(response)
